In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('punkt')

# 加载预训练的BERT模型和分词器
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 定义文本分类器
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# 示例Jira描述内容
description = """
Hi team,

Could you please help to look at the case described below to improve the
material report performance?

The query took 20 minutes or more to get the data for one month.

Open image-20240402-052748.png
image-20240402-052748.png
DB: medicalsanmhk-pts.42-q.com / PTSIII_E85

2024-04-01 02:48:24 EDT [31917]: [10-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 LOG:  duration: 501045.690 ms  execute <unnamed>: SELECT                            comp.created_time,comp.barcode,inc.incoming_number,inc.coo,pn.pn_id,pn.part_number,comp.total_qty,comp.current_qty,comp.lot_code,comp.remark,comp.component_id,usr.usr_name,comp.expire_time,aml.manufacturer_id,aml.manufacturer_pn,comp.pb_category, comp.status_id,comp.lpn,aml.aml_id,aml.status_id AS aml_status_id,aml.last_update,                                              inc.incoming_id,comp.manu_info_id,comp.date_code_str,comp.date_code,co.company_name,comp.msd_level,inc.supplier_id,com.company_name

                                                as supplier_name,com.vendor_code, comp.updated_date

                                                FROM

                                                part_number pn,component_1

                                                comp,aml,company

                                                co,user_info usr,incoming inc LEFT

                                                JOIN company com

                                                ON

                                                inc.supplier_id=com.company_id

                                                WHERE

                                                pn.pn_id=aml.pn_id

                                                AND

                                                aml.aml_id=inc.aml_id

                                                AND

                                                aml.manufacturer_id=co.company_id

                                                AND

                                                usr.usr_id=comp.usr_id

                                                AND

 inc.incoming_id=comp.incoming_id  AND comp.created_time > '1709251200'   AND comp.created_time < '1712016000'  ORDER BY comp.created_time DESC OFFSET 0 LIMIT 50000

2024-04-01 03:18:49 EDT [31913]: [10-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 ERROR: canceling statement due to user request

2024-04-01 03:18:49 EDT [31913]: [11-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 STATEMENT:   SELECT COUNT(*) AS total FROM ( SELECT                                 comp.created_time,comp.barcode,inc.incoming_number,inc.coo,pn.pn_id,pn.part_number,comp.total_qty,comp.current_qty,comp.lot_code,comp.remark,comp.component_id,usr.usr_name,comp.expire_time,aml.manufacturer_id,aml.manufacturer_pn,comp.pb_category, comp.status_id,comp.lpn,aml.aml_id,aml.status_id AS aml_status_id,aml.last_update,                                         inc.incoming_id,comp.manu_info_id,comp.date_code_str,comp.date_code,co.company_name,comp.msd_level,inc.supplier_id,com.company_name

                                                as supplier_name,com.vendor_code, comp.updated_date

                                                FROM

                                                part_number pn,component_1

                                                comp,aml,company

                                                co,user_info usr,incoming inc LEFT

                                                JOIN company com

                                                ON

                                                inc.supplier_id=com.company_id

                                                WHERE

                                                pn.pn_id=aml.pn_id

                                                AND

                                                aml.aml_id=inc.aml_id

                                                AND

                                                aml.manufacturer_id=co.company_id

                                                AND

                                                usr.usr_id=comp.usr_id

                                                AND

inc.incoming_id=comp.incoming_id  AND comp.created_time > '1709251200'   AND comp.created_time < '1712016000'  ORDER BY comp.created_time DESC ) AS totalSQLView
"""

# 分句处理
sentences = description.split('\n')

# 过滤掉无意义的句子
def filter_sentences(sentences, classifier):
    meaningful_sentences = []
    for sentence in sentences:
        if len(sentence.strip()) > 0:
            result = classifier(sentence)
            if result[0]['label'] == 'LABEL_1':  # 假设LABEL_1表示有意义的句子
                meaningful_sentences.append(sentence)
    return meaningful_sentences

# 过滤掉无意义的句子

for text in description.split('\n'):
    if len(text.strip()) > 0:
        result = classifier(text)
        print(text.strip())
        print(result)


# 过滤无意义的句子
#meaningful_sentences = filter_sentences(sentences, classifier)
#cleaned_description = ' '.join(meaningful_sentences)
#print("进一步清理后的描述内容：")
#print(cleaned_description)

# 提取关键词和特征
def extract_keywords(text):
    # 分词
    words = word_tokenize(text)
    
    # 移除停用词
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalnum()]
    
    return filtered_words

# 提取特征
#keywords = extract_keywords(cleaned_description)
#print("提取的关键词：")
#print(keywords)

# 使用TF-IDF向量化文本内容
#vectorizer = TfidfVectorizer()
#tfidf_matrix = vectorizer.fit_transform([cleaned_description])
#print("TF-IDF特征矩阵：")
#print(tfidf_matrix.toarray())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Hi team,
[{'label': 'LABEL_1', 'score': 0.40623027086257935}]
Could you please help to look at the case described below to improve the
[{'label': 'LABEL_1', 'score': 0.41702502965927124}]
material report performance?
[{'label': 'LABEL_1', 'score': 0.3863368034362793}]
The query took 20 minutes or more to get the data for one month.
[{'label': 'LABEL_1', 'score': 0.3764258921146393}]
Open image-20240402-052748.png
[{'label': 'LABEL_1', 'score': 0.3782578706741333}]
image-20240402-052748.png
[{'label': 'LABEL_1', 'score': 0.37823617458343506}]
DB: medicalsanmhk-pts.42-q.com / PTSIII_E85
[{'label': 'LABEL_1', 'score': 0.39695999026298523}]
2024-04-01 02:48:24 EDT [31917]: [10-1] db=PTSIII_E85,user=ptswriter,app=[unknown],client=10.191.8.52 LOG:  duration: 501045.690 ms  execute <unnamed>: SELECT                            comp.created_time,comp.barcode,inc.incoming_number,inc.coo,pn.pn_id,pn.part_number,comp.total_qty,comp.current_qty,comp.lot_code,comp.remark,comp.component_id,usr.usr_na